## Classic Neural Network for entanglement detection

In this notebook we are going to use fully connected neural network for the task of entanglement detection using quantum states.\
The dataset is imported with pandas and the network is implemented with the torch library.


In [22]:
# Import the required libraries


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from tqdm.auto import trange # for the loading bar in for loop


### Dataset

We use the dataset of the quantum state: the ```ds_haar_op```.
It's composed by 16 features and 1 label, 0 or 1 if the state is entangled or separable.\
The 16 features represent the entries of the unitary matrix that generate the state composed by two qubits: $\ket{\psi_i} = U_i \ket{00} \, \, i = 1,..,N$ where $N$ is the number of data.\
Then the denisty matrices are used for the training. They are computed as outer produtct of the states

$$ \rho_i = \ket{\psi_i}\bra{\psi_i}.$$

Then the dataset is split in train and test part using the hypeparameters define below. 


In [23]:
# Hyperparameters value

N_DATA = 2000
N_TRAIN = 1600
N_TEST = N_DATA - N_TRAIN

BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.001


dataset_U = pd.read_csv("../datasets/ds_haar_op.csv") # import the dataset
dataset_U = dataset_U[:N_DATA] # select the number of data

# Separate features (X) and labels (y)

# Drop the 17th column that is the label
X = dataset_U.drop(columns=dataset_U.columns[16])

y = dataset_U.iloc[:, 16]  # Assuming the label is in the 17th column (index 16)
y = th.tensor(y.to_numpy(dtype=int)) # convert the labels in torch tensor

dataset_psi_out = X.iloc[:, :4] # select only the first row of the unitary matrix, because it represent the output state after apply U to |00>
dataset_psi_out_np = dataset_psi_out.to_numpy(dtype=np.csingle) # convert the dataset in numpy

# compute the density matrices dataset as a torch tensor 
dens_matrices = th.tensor(np.array([(np.outer(dataset_psi_out_np[i], np.conj(dataset_psi_out_np[i]))).flatten() for i in range(N_DATA)])) 

# divide in real and imaginary part
dens_matrices_real_imag = th.cat((th.real(dens_matrices), th.imag(dens_matrices)), dim = 1)

# divide in test train part

#train
dens_matrices_train = dens_matrices_real_imag[:N_TRAIN]
y_train = y[:N_TRAIN]

train_mapped_dataset = TensorDataset(dens_matrices_train,y_train) # create dataset
train_mapped_loader = DataLoader(train_mapped_dataset, shuffle=True, batch_size=BATCH_SIZE) # create dataloader for the training


#TEST
dens_matrices_test = dens_matrices_real_imag[N_TRAIN:N_DATA]
y_test = y[N_TRAIN:N_DATA]

test_mapped_dataset = TensorDataset(dens_matrices_test,y_test) # create your datset
test_mapped_loader = DataLoader(test_mapped_dataset, shuffle=False, batch_size=BATCH_SIZE) # create dataloader for the test


### Model

Define the model as an inherint class from the nn.Module of pytorch.
It's define the constructor, i.e. the architecture of the network with:
* input layer 
* hidden layer 
* output layer

Then is the define a function for the forward part of the training in which is applied an activaction function to each layer and return the ouput of the neural network

In [24]:
# Define the neural network model through a class:

class Net_cluster1(nn.Module):

    # define the model with the constructor of the class
    def __init__(self):
        super(Net_cluster1, self).__init__() 
        self.l1 = nn.Linear(in_features = 16+16 , out_features = 50) # input layer
        self.l2 = nn.Linear(in_features = 50, out_features = 15) # hidden layer
        self.term = nn.Linear(in_features = 15, out_features = 2) # output layer

    # this function is used for the forward training part
    def forward(self, x: th.Tensor) -> th.Tensor:
        
        x : th.Tensor = x.flatten(start_dim=1)
        x : th.Tensor = F.relu(self.l1(x))
        x : th.Tensor  = F.relu(self.l2(x))
        #x : Tensor = F.relu(self.l3(x))
        logits : th.Tensor = self.term(x)
        out: th.Tensor = F.softmax(input=logits, dim=1)

        return out

### Important function

There are some fundamental function below for the learning of the neural network

In [25]:
def get_batch_accuracy(logit, target):
    """
    Obtain accuracy for one batch of data
    Input:
        - logit(torch.tensor): The predictions from the model 
        - target(torch.tensor): The y true values 
    Return:
        - accuracy(float): The value of the accuracy
    
    """
    corrects = (th.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects / target.size(0)
    return accuracy.item()


def train_loop(model, train_loader, EPOCHS, optimizer, criterion):
    """
    Function for training the model.
    Input:
        - model(torch model): The neural network
        - train_loader(torch DataLoader): The train dataset passed as a torch DataLoader
        - EPOCHS(int): Number of epochs for the training
        - optimizer(torch optimizer): Optimizer for the learning algorithm
        - criterion(torch loss functions): The loss function for the learning algorithm
    Return: Print the loss and train accuracy and train the model
    """

    # do the training loop through the number of epochs
    for epoch in trange(EPOCHS):
        train_running_loss = 0.0
        train_acc = 0.0

        model = model.train()  # Set the model to training mode: relevant for dropout, batchnorm, etc.

        # Actual (batch-wise) training step
        for i, (images, labels) in enumerate(train_loader):
            # Forward pass + (automated) BackProp + Loss computation
            logits = model(images)
            loss = criterion(logits, labels) # calculate the loss

            optimizer.zero_grad()  # Reset the gradients to zero: otherwise they accumulate!
            loss.backward()  # Backpropagation

            # Update model params
            optimizer.step()

            train_running_loss += loss.detach().item()
            train_acc += get_batch_accuracy(logits, labels)

        model.eval()

        print(f"Epoch: {epoch+1} | Loss: {train_running_loss/i} | Train Accuracy: {train_acc/i}")


### Trainig and test part 

Here some important things for the training are defined:
* The model
* The loss function
* The optimizer

Then is used the train_loop function for the training part and after is calculated the test accuracy

In [26]:
model = Net_cluster1() # define the model as an object of the model class
criterion = nn.CrossEntropyLoss()  # Loss function
optimizer = th.optim.Adam(params=model.parameters(), lr=LEARNING_RATE) # the optimizer

In [27]:

train_loop(model, train_mapped_loader, EPOCHS, optimizer, criterion) # training part
print("")

# calculate the test accuracy
test_acc = 0.0
for i, (dens_matr, labels) in enumerate(test_mapped_loader):
    outputs = model(dens_matr)
    test_acc += get_batch_accuracy(outputs, labels)

print(f"Test Accuracy: {test_acc/i}")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | Loss: 0.7067818921439502 | Train Accuracy: 51.7219387755102
Epoch: 2 | Loss: 0.7036765546214824 | Train Accuracy: 51.7219387755102
Epoch: 3 | Loss: 0.6985502389012551 | Train Accuracy: 59.05612244897959
Epoch: 4 | Loss: 0.6897046152426272 | Train Accuracy: 61.0969387755102
Epoch: 5 | Loss: 0.6757097560532239 | Train Accuracy: 65.88010204081633
Epoch: 6 | Loss: 0.6532498926532512 | Train Accuracy: 71.30102040816327
Epoch: 7 | Loss: 0.6227319946094435 | Train Accuracy: 76.08418367346938
Epoch: 8 | Loss: 0.588756433555058 | Train Accuracy: 81.12244897959184
Epoch: 9 | Loss: 0.5584424715869281 | Train Accuracy: 83.6734693877551
Epoch: 10 | Loss: 0.5313011274045828 | Train Accuracy: 86.4795918367347

Test Accuracy: 91.66666666666667


### Plot